In [28]:
import win32pdhutil
import win32pdh
import time

In [29]:
def GetPerformanceAttributes(object, counter, instance = None, inum=-1,
                             format = win32pdh.PDH_FMT_LONG, machine=None):
    # NOTE: Many counters require 2 samples to give accurate results,
    # including "% Processor Time" (as by definition, at any instant, a
    # thread's CPU usage is either 0 or 100).  To read counters like this,
    # you should copy this function, but keep the counter open, and call
    # CollectQueryData() each time you need to know.
    # See http://support.microsoft.com/default.aspx?scid=kb;EN-US;q262938
    # and http://msdn.microsoft.com/library/en-us/dnperfmo/html/perfmonpt2.asp
    # My older explanation for this was that the "AddCounter" process forced
    # the CPU to 100%, but the above makes more sense :)
    path = win32pdh.MakeCounterPath( (machine,object,instance, None, inum,counter) )
    hq = win32pdh.OpenQuery()
    try:
        hc = win32pdh.AddCounter(hq, path)
        try:
            win32pdh.CollectQueryData(hq)
            time.sleep(1)
            win32pdh.CollectQueryData(hq)
            type, val = win32pdh.GetFormattedCounterValue(hc, format)
            return val
        finally:
            win32pdh.RemoveCounter(hc)
    finally:
        win32pdh.CloseQuery(hq)

In [30]:
import os
os.environ['http_proxy'] = "http://proxy-chain.intel.com:911" 
os.environ['https_proxy'] = "http://proxy-chain.intel.com:912" 


In [31]:
%pip install plotly ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [32]:
import plotly.graph_objects as go
from IPython.display import display, clear_output
import time
from datetime import datetime
import random

# These are placeholders. Replace with your actual function calls.
# def GetPerformanceAttributes(category, counter, instance):
#     if counter == "% Idle Time":
#         return random.random() * 100
#     elif counter == "Interrupts/sec":
#         return random.randint(0, 1000)
#     else:
#         return None

x, y1, y2 = [], [], []

for i in range(100):
    time.sleep(0.1)  # simulate time delay

    # Add new data
    y1.append(GetPerformanceAttributes("Processor", "% Idle Time", "_Total"))
    y2.append(GetPerformanceAttributes("Processor", "Interrupts/sec", "_Total"))
    x.append(datetime.now())
    
    # Clear current figure
    clear_output(wait=True)
    
    # Create new figure and display it
    fig = go.Figure()

    # Add traces
    fig.add_trace(go.Scatter(x=x, y=y1, mode='lines+markers', name='% Idle Time'))
    fig.add_trace(go.Scatter(x=x, y=y2, mode='lines+markers', name='Interrupts/sec', yaxis='y2'))

    # Create axis objects
    fig.update_layout(
        xaxis=dict(domain=[0.3, 0.7]),
        yaxis=dict(title='% Idle Time', titlefont=dict(color='blue'), tickfont=dict(color='blue')),
        yaxis2=dict(title='Interrupts/sec', titlefont=dict(color='red'), tickfont=dict(color='red'), anchor='free', overlaying='y', side='right')
    )

    display(fig)


In [34]:
import plotly.graph_objects as go
from IPython.display import display, clear_output
import time
from datetime import datetime
import random

# These are placeholders. Replace with your actual function calls.
# def GetPerformanceAttributes(category, counter, instance):
#     if counter == "% Idle Time":
#         return random.random() * 100
#     elif counter == "Interrupts/sec":
#         return random.randint(0, 1000)
#     else:
#         return None

x, y1, y2, y3, y4 = [], [], []
y1_sum, y2_sum, y3_sum, y4_sum = 0.0, 0.0, 0.0, 0.0

for i in range(100):
    # time.sleep(0.1)  # simulate time delay

    # Add new data
    y1_value = GetPerformanceAttributes("Processor", "% Idle Time", "_Total")
    y2_value = GetPerformanceAttributes("Processor", "Interrupts/sec", "_Total")
    y1.append(y1_value)
    y2.append(y2_value)
    x.append(datetime.now())

    y1_sum += y1_value
    y2_sum += y2_value

    # If we have more than 100 data points, subtract the oldest one from the sum
    if len(y1) > 100:
        y1_sum -= y1.pop(0)
        y2_sum -= y2.pop(0)
        x.pop(0)

    y1_avg = y1_sum / len(y1)
    y2_avg = y2_sum / len(y2)

    # Clear current figure
    clear_output(wait=True)
    
    # Create new figure and display it
    fig = go.Figure()

    # Add traces
    fig.add_trace(go.Scatter(x=x, y=y1, mode='lines+markers', name='% Idle Time'))
    fig.add_trace(go.Scatter(x=x, y=y2, mode='lines+markers', name='Interrupts/sec', yaxis='y2'))

    # Create axis objects
    fig.update_layout(
        xaxis=dict(domain=[0.3, 0.7]),
        yaxis=dict(title='% Idle Time', titlefont=dict(color='blue'), tickfont=dict(color='blue')),
        yaxis2=dict(title='Interrupts/sec', titlefont=dict(color='red'), tickfont=dict(color='red'), anchor='free', overlaying='y', side='right'),
        annotations=[
            dict(x=0.01, y=1.06, xref='paper', yref='paper', text=f'Average % Idle Time (last 10s): {y1_avg:.2f}', showarrow=False, font=dict(color='blue')),
            dict(x=0.01, y=1.02, xref='paper', yref='paper', text=f'Average Interrupts/sec (last 10s): {y2_avg:.2f}', showarrow=False, font=dict(color='red')),
        ]
    )

    display(fig)
